## plot for Hyperparameter Selection
You should run Hyperparameter.sh firat

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi
from cycler import cycler
import torch
import torch.nn.functional as F
import torch.nn as nn

from robustbench.utils import load_model

from core.attacks.utils import clamp, batch_multiply
from sampler import Sampler
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import numpy as np


from matplotlib import cm
from matplotlib import axes

In [ ]:
def get_data_eval(a='pgdce40',n='clean',m='Langevin',topk=10):
    basefile='./csv-out/ResNet18-linf-{}-Sce{}-eval.csv'
    res = []
    for i in range(2,21,2):
        data = pd.read_csv(basefile.format(a,i))
        nums=len(data)
        s_nums=nums//topk
        if i==2:
            print('clean x acc:',sum(data['y_clean_label']==data["ture_label"])/nums)
            print('adv x acc:',sum(data['y_adv_label']==data["ture_label"])/nums)
        
        y_prob = data['y_{}_{}_prob'.format(m,n)].to_numpy().reshape(-1,topk)
        true_label = data.ture_label.to_numpy().reshape(-1,topk)[:,0]
        ysl = data['y_{}_sample_label'.format(n)].to_numpy().reshape(-1,topk)
        temp = []
        for k in range(topk):
            pre_label = np.array([ysl[i][j] for i,j in zip(range(len(ysl)),y_prob[:,0:k+1].argmax(axis=1))])
            acc = sum(pre_label==true_label)/s_nums
            temp.append(round(acc*100.,2))
        res.append(temp)
    return res

def get_data_agg(a='pgdce40',n='clean',m='Langevin',topk=10):
    basefile='./csv-out/ResNet18-linf-{}-Sce{}-agg.pth'
    res = []
    ind = {'clean':'cN','adv':'aN'}
    for i in range(2,21,2):
        data = torch.load(basefile.format(a,i))
        print(data[ind[n]])
        temp=[]
        for k in range(topk):
            temp.append(round(data[ind[n]][k+1]/100,2))
        res.append(temp)
    return res

def plot_heat_agg(a='pgdce40',n='clean',m='Langevin',topk=10):
    data = get_data_agg(a=a,n=n,m=m,topk=topk)
    
    fig = plt.figure(figsize=(8, 6))
    plt.rc('font',family='Times New Roman')
    ax = fig.add_subplot(111)
    
    ax.set_yticks(range(10))
    ax.set_yticklabels([i for i in range(2,21,2)])
    ax.set_ylabel("Sampler iters n",fontsize=16) 
    
    ax.set_xticks(range(topk))
    ax.set_xticklabels([i+1 for i in range(10)])
    ax.set_xlabel("top-k",fontsize=16)
    
    im = ax.imshow(data, cmap='rainbow',alpha=0.8,origin='lower')
    plt.colorbar(im)
    for i in range(topk):
        for j in range(10):
            text = plt.text(j, i, data[i][j],
                           ha="center", va="center", color="black", fontsize=10)
    plt.savefig('./pic/{}-{}-MBC.png'.format(a,n),bbox_inches='tight')
    plt.show()

    
def plot_heat(a='pgdce40',n='clean',m='Langevin',topk=10):
    data = get_data_eval(a=a,n=n,m=m,topk=topk)
    
    fig = plt.figure(figsize=(8, 6))
    plt.rc('font',family='Times New Roman')
    ax = fig.add_subplot(111)
    
    ax.set_yticks(range(10))
    ax.set_yticklabels([i for i in range(2,21,2)])
    ax.set_ylabel("Sampler iters n",fontsize=16) 
#     plt.yticks(fontsize=fontsize)
    
    ax.set_xticks(range(topk))
    ax.set_xticklabels([i+1 for i in range(10)])
    ax.set_xlabel("top-k",fontsize=16)
    
    im = ax.imshow(data, cmap='rainbow',alpha=0.8,origin='lower',)
    plt.colorbar(im)
    
    for i in range(topk):
        for j in range(10):
            text = plt.text(j, i, data[i][j],
                           ha="center", va="center", color="black", fontsize=10)
    plt.savefig('./pic/{}-{}-MES.png'.format(a,n),bbox_inches='tight')
    plt.show()

In [2]:
plot_heat(a='pgdce40',n='adv',topk=10)
plot_heat(a='pgdcw40',n='adv',topk=10)
plot_heat(a='pgdcw40',n='clean',topk=10)
plot_heat_agg(a='pgdcw40',n='clean',topk=10)
plot_heat_agg(a='pgdcw40',n='adv',topk=10)
plot_heat_agg(a='pgdce40',n='adv',topk=10)

NameError: name 'plot_heat' is not defined

## plot $ (x,y) \sim p_{\theta}(x,y)$

In [ ]:
x = torch.randn(20,3,32,32)
x = clamp(x,0,1)

plt.figure()
plt.imshow(make_grid(x,nrow=5).permute(1,2,0))
plt.axis('off')
plt.savefig('./pic/initialize.png')

In [3]:
from models import ResNet18

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from models import ResNet18

model_test = ResNet18().to(device)

checkpoint = torch.load('./checkpoint/resnet18.pth')
model_test = nn.DataParallel(model_test).to(device) # put this line after loading state_dict if the weights are saved without module.
if 'state_dict' in checkpoint.keys():
    model_test.load_state_dict(checkpoint['state_dict'])
else:
    model_test.load_state_dict(checkpoint)

model_test.float()
model_test.eval()
# model = nn.DataParallel(model).cuda()

cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2471, 0.2435, 0.2616)
mu = torch.tensor(cifar10_mean).view(3,1,1).cuda()
std = torch.tensor(cifar10_std).view(3,1,1).cuda()

def normalize_PGDAT(X):
    return (X - mu)/std

# def normalize_PGDAT(X):
#     return X

class normalize_model():
    def __init__(self, model):
        self.model_test = model
    def __call__(self, x):
        return self.model_test(normalize_PGDAT(x))
model = normalize_model(model_test)
del checkpoint

NameError: name 'torch' is not defined

In [ ]:
# sample for resnet
se = Sampler(model,loss='ce',grad_norm='linf',eps=8/255,Lnoise_eps=0.01)

y=torch.LongTensor([i for i in range(10)]*2)

print('sample label',y)
x_adv,y = x.to(device),y.to(device)
x_adv,_=se.sample_multi(x_adv,y,n_iter=500)


plt.figure()
plt.imshow(make_grid(x_adv.cpu(),nrow=5).permute(1,2,0))
plt.axis('off')
y_pre = model(x_adv.detach())
plt.savefig('./pic/resnet18.png',bbox_inches='tight')

In [ ]:
model = load_model(model_name='Debenedetti2022Light_XCiT-S12', dataset='cifar10', threat_model='Linf').to(device)

# sample for XCiT
se = Sampler(model,loss='ce',grad_norm='linf',eps=8/255,Lnoise_eps=0.01)

y=torch.LongTensor([i for i in range(10)]*2)

print('sample label',y)
x_adv,y = x.to(device),y.to(device)
x_adv,_=se.sample_multi(x_adv,y,n_iter=500)


plt.figure()
plt.imshow(make_grid(x_adv.cpu(),nrow=5).permute(1,2,0))
plt.axis('off')
y_pre = model(x_adv.detach())
plt.savefig('./pic/XCiT.png',bbox_inches='tight')

## For Table 1 (Full gradient test)

In [ ]:
upper_limit, lower_limit = 1, 0

def clamp(X, lower_limit, upper_limit):
    return torch.max(torch.min(X, upper_limit), lower_limit)

def attack_pgd(model, X, y, epsilon, alpha, attack_iters, restarts, use_CWloss=False,):
    max_loss = torch.zeros(y.shape[0]).cuda()
    max_delta = torch.zeros_like(X).cuda()
    for _ in range(restarts):
        delta = torch.zeros_like(X).cuda()
        delta.uniform_(-epsilon, epsilon)
        delta.data = clamp(delta, lower_limit - X, upper_limit - X)
        delta.requires_grad = True
        for _ in range(attack_iters):
            output = model(X + delta)
            index = torch.where(output.max(1)[1] == y)
            if len(index[0]) == 0:
                break
            if use_CWloss:
                loss = CW_loss(output, y)
            else:
                loss = F.cross_entropy(output, y)
            loss.backward()
            grad = delta.grad.detach()
            d = delta[index[0], :, :, :]
            g = grad[index[0], :, :, :]
            d = torch.clamp(d + alpha * torch.sign(g), -epsilon, epsilon)
            d = clamp(d, lower_limit - X[index[0], :, :, :], upper_limit - X[index[0], :, :, :])
            delta.data[index[0], :, :, :] = d
            delta.grad.zero_()
        all_loss = F.cross_entropy(model(X + delta), y, reduction='none').detach()
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)
    return max_delta

def CW_loss(x, y):
    x_sorted, ind_sorted = x.sort(dim=1)
    ind = (ind_sorted[:, -1] == y).float()
    
    loss_value = -(x[np.arange(x.shape[0]), y] - x_sorted[:, -2] * ind - x_sorted[:, -1] * (1. - ind))
    return loss_value.mean()

def evaluate_pgd(test_loader, model, attack_iters, restarts, eps=8, step=2, use_CWloss=False):
    epsilon = eps / 255.
    alpha = step / 255.
    pgd_loss = 0
    pgd_acc = 0
    n = 0
    model.eval()
    for i, (X, y) in enumerate(test_loader):
        X, y = X.cuda(), y.cuda()
        pgd_delta = attack_pgd(model, X, y, epsilon, alpha, attack_iters, restarts, use_CWloss=use_CWloss)
#         with torch.no_grad():
        output = model(X + pgd_delta)
        loss = F.cross_entropy(output, y)
        pgd_loss += loss.item() * y.size(0)
        pgd_acc += (output.max(1)[1] == y).sum().item()
        n += y.size(0)
    return pgd_loss/n, pgd_acc/n

transformer_test  = transforms.Compose([transforms.ToTensor()])

data_loader  = torch.utils.data.DataLoader(datasets.CIFAR10('../data', train=False,
                                                        transform=transformer_test, download=True),
                                    batch_size=64, shuffle=False, num_workers=10)

In [ ]:
from CorrectionModel import CModel

#model:your trained model
Cmodel = CModel(model,topk=2,n_iter=5)

In [ ]:
_, pgd_acc_CE = evaluate_pgd(data_loader, Cmodel, attack_iters=40, restarts=2, eps=8, step=2, use_CWloss=False,)
print('PGD-40 (2 restarts, step 2, CE loss) acc: ', pgd_acc_CE)

### Evaluate PGD (CW loss) acc ###
_, pgd_acc_CW = evaluate_pgd(data_loader, Cmodel, attack_iters=40, restarts=2, eps=8, step=2, use_CWloss=True)
print('PGD-40 (2 restarts, step 2, CW loss) acc: ', pgd_acc_CW)

## Analyze and process the output statistics files

Before this, you should run all attacks ['PGD40', 'CW40', 'APGD-ce','APGD-dlr',"fab",'square'] under a model

for test detail see test.sh

In [ ]:
def get_MES_adv_acc(data,k,topk=5):
    nums=len(data)
    s_nums=nums//topk
    y_prob = data['ES_Langevin_adv'].to_numpy().reshape(-1,topk)
    true_label = data.ture_label.to_numpy().reshape(-1,topk)[:,0]
    ysl = data['y_adv_sample_label'].to_numpy().reshape(-1,topk)
    
    pre_label = np.array([ysl[i][j] for i,j in zip(range(len(ysl)),y_prob[:,0:k].argmin(axis=1))])
    acc = sum(pre_label==true_label)/s_nums
    return round(acc*100.,2)
    
def get_MES_clean_acc(data,k,topk=5):
    nums=len(data)
    s_nums=nums//topk
    y_prob = data['ES_Langevin_clean'].to_numpy().reshape(-1,topk)
    true_label = data.ture_label.to_numpy().reshape(-1,topk)[:,0]
    ysl = data['y_clean_sample_label'].to_numpy().reshape(-1,topk)
    
    pre_label = np.array([ysl[i][j] for i,j in zip(range(len(ysl)),y_prob[:,0:k].argmin(axis=1))])
    acc = sum(pre_label==true_label)/s_nums
    return round(acc*100.,2)
    
def get_MBC_adv_acc(data,k,n):
    return round(data['aN'][k]/n,2)

def get_MBC_clean_acc(data,k,n):
    return round(data['cN'][k]/n,2)
    
def batch_pro_csv(model):
    res = pd.DataFrame(columns=['Clean','PGD40', 'CW40', 'APGD-ce','APGD-dlr',"fab",'square'],index=[i for i in range(11)])
    attack = ['pgdce40','pgdcw40','apgdce100','apgddlr100','fabce100','squarece100']
    basecsv='./csv-out/{}-linf-{}-Sce10-eval.csv'
    baseagg='./csv-out/{}-linf-{}-Sce10-agg.pth'
    for i,a in enumerate(attack):
        csvdata =pd.read_csv(basecsv.format(model,a))
        aggdata = torch.load(baseagg.format(model,a))
        nums=len(data)
        s_nums=nums//topk
        cleanacc=sum(csvdata['y_clean_label']==csvdata["ture_label"])/len(csvdata)
        advacc =sum(csvdata['y_adv_label']==csvdata["ture_label"])/len(csvdata)
        res.iloc[0][0] = round(cleanacc*100,2)
        res.iloc[0][i+1] = round(advacc*100,2)
        for j in range(1,11):
            k=((j-1)//2)+1
            if j%2:
                ca = get_MES_clean_acc(csvdata,k)
                aa = get_MES_adv_acc(csvdata,k)
            else:
                ca = get_MBC_clean_acc(aggdata,k,s_nums)
                aa = get_MBC_adv_acc(aggdata,k,s_nums)
            res.iloc[j][0] = ca            
            res.iloc[j][i+1] = aa
    res['top-k']=[0,1,1,2,2,3,3,4,4,5,5]
    res.to_csv('./csv-out/pro/{}.csv'.format(model))

In [ ]:
#example
batch_pro_csv('cifar100-Debenedetti2022Light_XCiT-M12')